In [1]:
!pip install "opencv-python-headless<4.3"
!pip install cellpose

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.5 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 3.4.18.65 which is incompatible.
albucore 0.0.23 requires opencv-python-headless>=4.9.0.80, but you have opencv-python-headless 3.4.18.65 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [2]:
!nvcc --version
!nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Mon Mar 10 11:08:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8       

In [3]:
import numpy as np
import tifffile
from cellpose import models
import os
import glob
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Projected/Static-A-1/Nuclei'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Nuclei'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Define Cellpose parameters
model_type = 'cyto3'  # Using cyto3 model for nuclei
channels = [0, 0]     # Single channel mode (no nuclear channel needed since entire image is nuclei)
diameter = 30         # Approximate size of nuclei
cellprob_threshold = 0.0  # Threshold for cell probability
flow_threshold = 0.4      # Threshold for flows
resample = False      # Don't resample dynamics
do_3D = False         # 2D processing
stitch_threshold = 0  # No stitching needed for 2D

# Create Cellpose model
print("Creating Cellpose model...")
CP = models.CellposeModel(model_type=model_type, gpu=True)

# Find all .tif files in the input directory
input_files = glob.glob(os.path.join(input_dir, '*.tif'))
print(f"Found {len(input_files)} .tif files to process")

# Process each file
for input_file in input_files:
    # Get base filename for output
    filename = os.path.basename(input_file)
    base_name = os.path.splitext(filename)[0]
    output_file = os.path.join(output_dir, f"{base_name}_mask.tif")

    print(f"Processing: {filename}")

    # Load the image
    img = tifffile.imread(input_file)
    print(f"  Image shape: {img.shape}")

    # For 2D single-channel images, add dimensions to match what Cellpose expects
    if len(img.shape) == 2:
        # This is a 2D single-channel image (H, W)
        image = img[np.newaxis, ..., np.newaxis]  # Add z and channel dimensions (z, y, x, c)
    else:
        raise ValueError(f"Unexpected image shape: {img.shape}. Expected 2D image with shape (H, W).")

    # Run segmentation
    print(f"  Running segmentation with diameter={diameter}, cellprob_threshold={cellprob_threshold}, flow_threshold={flow_threshold}")
    masks, flows, _ = CP.eval(image,
                              channels=channels,
                              diameter=diameter,
                              resample=resample,
                              cellprob_threshold=cellprob_threshold,
                              flow_threshold=flow_threshold,
                              do_3D=do_3D,
                              stitch_threshold=stitch_threshold)

    # Process the masks
    masks = masks.squeeze()  # Remove any extra dimensions
    print(f"  Segmentation complete. Found {len(np.unique(masks))-1} objects")

    # Save the masks
    tifffile.imwrite(output_file, masks.astype(np.uint32))
    print(f"  Saved masks to {output_file}")

print("All processing complete!")

Mounted at /content/drive
Creating Cellpose model...


100%|██████████| 25.3M/25.3M [00:00<00:00, 30.6MB/s]


Found 16 .tif files to process
Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Nuclei_contrast_bg_tophat.tif
  Image shape: (1024, 1024)
  Running segmentation with diameter=30, cellprob_threshold=0.0, flow_threshold=0.4
  Segmentation complete. Found 325 objects
  Saved masks to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Nuclei/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq001_Nuclei_contrast_bg_tophat_mask.tif
Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Nuclei_contrast_bg_tophat.tif
  Image shape: (1024, 1024)
  Running segmentation with diameter=30, cellprob_threshold=0.0, flow_threshold=0.4
  Segmentation complete. Found 256 objects
  Saved masks to /content/drive/MyDrive/knowledge/University/Master/Thesis/Segmented/Static-A-1/Nuclei/denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq002_Nuclei_contrast_bg_tophat_mask.tif
Processing: denoised_0Pa_A1_19dec21_20xA_L2RA_FlatA_seq003_Nuclei_contrast_bg_tophat.tif
  Image shape: (1024, 